In [17]:
import numpy as np
import pandas as pd
import polars as pl

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
base_stories_df = pd.read_csv(r"C:\GitHub\Data-Analysis_competition\Data\base_stories.tsv" , sep='\t')
fiction_stories_test_df = pd.read_csv(r"C:\GitHub\Data-Analysis_competition\Data\fiction_stories_test.tsv", sep='\t')
fiction_stories_practice_df = pd.read_csv(r"C:\GitHub\Data-Analysis_competition\Data\fiction_stories_practice.tsv", sep='\t')

In [28]:
print(base_stories_df.columns)
print(fiction_stories_test_df.columns)
print(fiction_stories_practice_df.columns)

Index(['id', 'category', 'title', 'story'], dtype='object')
Index(['id', 'story'], dtype='object')
Index(['id_a', 'id_b', 'title_a', 'title_b', 'story'], dtype='object')


In [29]:
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))

In [ ]:
all_text = pd.concat([base_stories_df['story'], fiction_stories_test_df['story'], fiction_stories_practice_df['story']])
vectorizer.fit(all_text)

TfidfVectorizer(analyzer='char', ngram_range=(2, 3))

In [31]:
base_vectors = vectorizer.transform(base_stories_df['story'])
test_vectors = vectorizer.transform(fiction_stories_test_df['story'])

In [32]:
sim_matrix = cosine_similarity(test_vectors, base_vectors)

In [33]:
top2_indices = np.argsort(-sim_matrix, axis=1)[:, :2]

In [34]:
base_ids = base_stories_df['id'].values
predictions = []

for i in range(len(fiction_stories_test_df)):
    idx_a, idx_b = top2_indices[i]
    # 小さい方のIDをid_a、大きい方をid_bとする（一般的慣習）
    ids = sorted([base_ids[idx_a], base_ids[idx_b]])
    predictions.append([fiction_stories_test_df.iloc[i]['id'], ids[0], ids[1]])

In [35]:
submission_df = pd.DataFrame(predictions, columns=['id', 'id_a', 'id_b'])
submission_df.to_csv('C:\GitHub\Data-Analysis_competition\Analysis\sigante_anime\output\output.csv', index=False)

<>:2: SyntaxWarning: invalid escape sequence '\G'
<>:2: SyntaxWarning: invalid escape sequence '\G'
C:\Users\sk062\AppData\Local\Temp\ipykernel_16708\520861881.py:2: SyntaxWarning: invalid escape sequence '\G'
  submission_df.to_csv('C:\GitHub\Data-Analysis_competition\Analysis\sigante_anime\output\output.csv', index=False)


In [36]:
# 練習用データ（fiction_stories_practice_df）を用いて精度を算出する
practice_vectors = vectorizer.transform(fiction_stories_practice_df['story'])
sim_matrix_practice = cosine_similarity(practice_vectors, base_vectors)
top2_indices_practice = np.argsort(-sim_matrix_practice, axis=1)[:, :2]

correct_count = 0
for i in range(len(fiction_stories_practice_df)):
    # 正解のIDセット
    true_ids = {fiction_stories_practice_df.iloc[i]['id_a'], fiction_stories_practice_df.iloc[i]['id_b']}
    
    # 予測されたIDセット
    idx_a, idx_b = top2_indices_practice[i]
    pred_ids = {base_ids[idx_a], base_ids[idx_b]}
    
    # 両方のIDが一致しているか確認
    if true_ids == pred_ids:
        correct_count += 1

In [37]:
accuracy = correct_count / len(fiction_stories_practice_df)

In [38]:
print(f"Practice Data Accuracy: {accuracy:.4f} ({correct_count} / {len(fiction_stories_practice_df)})")

Practice Data Accuracy: 0.3000 (6 / 20)
